In [1]:
import scipy.stats as stats
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

# Assessment Study Notes Wk3

### 1. Docker
* *Finished assignment*
* **TODO: Review & try extra credit?**

### 2. AWS
* *Finished assignment*

### 3. SQL
### 4. Python SQL
### 5. Mongo DB
### 6. Web Scraping
### 7. Spark RDDs
### 8. Spark SQL

## 1. Docker
> Docker is a platform for developers and sysadmins to **develop, deploy, and run** applications with containers.

* **Docker vs Vagrant**: Docker doesn't have OS inside container, just sits on top...*(confirm)*
* **image** includes everything needed to run an app
* **container** is a runtime instance of an image



```
# list containers, images
docker container ls
docker image ls

# container removal
docker stop (container-name)
docker container rm (container-name)

# or force removal without stopping
docker container rm (container) -f

# flags
-d #detached, to keep container even upon exit
-p #port, example: "-p 8081:80", to interact with

exit #to exit
```
#### Map Folder with Volumes
```
# Let’s try it for nginx
dck run -d -p 8085:80 --name my-nginx-2 -v $(pwd):/usr/share/nginx/html nginx 
# make file in $(pwd) after adding volume in container
touch index.html
# then check inside container
dck container exec -it my-nginx-2 bash
cd /usr/share/nginx/html
ls # should now show sahred file index.html
```

#### Other Useful Commands
```

```

## 2. AWS

In [2]:
import boto3
s3 = boto3.client('s3')

s3.upload_file(Filename=local_file_name, 
               Bucket=bucket_name, 
               Key=remote_file_name)

response = s3.get_object(Bucket=bucket_name,
                         Key=remote_file_name)

s3.download_file(Bucket=bucket_name,
                 Key=remote_file_name,
                 Filename="downloaded-" + local_file_name)

NameError: name 'local_file_name' is not defined

## 3. SQL
* Run .sql file:
```
postgres-# \i 'home/data/lecture_create.sql'
```
* Load a file from CSV:
```
postgres=# COPY your_table FROM '/home/data/your_table.csv' DELIMITER ',';
```
```
docker run --name pgserv -d -p 5432:5432 -v "$PWD":/home/data -e POSTGRES_PASSWORD='pass' postgres
docker start pgserv
# Start with bash
docker exec -it pgserv bash
# Start with postgres
docker exec -it pgserv psql -U postgres
```
```
\d #to list tables
\d [tablename] #to list columns in [tablename]

```

**Postgres Setup - Chelsea**
```
$ docker run --name pgserv -d -p 5432:5432 -v "$PWD":/home/data -e POSTGRES_PASSWORD='pass' postgres
$ docker start pgserv
$ docker exec -it pgserv psql -U postgres

postgres=# CREATE DATABASE [dbname]; # <-- fill in [dbname] without []

$ docker exec -it pgserv bash
# psql -U postgres [dbname] < [file.sql]
# psql -U postgres [dbname]
```
**Postgres Setup - Chelsea**


## 4. Python SQL
## General Workflow

1. Establish connection to Postgres database using psycopg2
2. Create a cursor
3. Use the cursor to execute SQL queries and retrieve data
4. Commit SQL actions
4. Close the cursor and connection
```
import psycopg2
conn = psycopg2.connect(user='postgres', host='localhost', port='5432', password='pass')
cur = conn.cursor()
--queries--
query = '''
        CREATE TABLE golf (
            Date timestamp,
            Outlook varchar(10),
            Temperature integer, 
            Humidity integer, 
            Windy boolean,
            Result varchar
        );
        '''
cur.execute(query)
--queries--
conn.commit()
conn.close()
```

# Key Things to Remember

* Connections must be established using an existing database, username, database IP/URL, and maybe passwords
* If you have no created databases, you can connect to Postgres using the dbname 'postgres' to initialize db commands
* Data changes are not actually stored until you choose to commit. This can be done either through `conn.commit()` or setting `autocommit = True`.  Until commited, all transactions is only temporary stored.
* Autocommit = True is necessary to do database commands like CREATE DATABASE.  This is because Postgres does not have temporary transactions at the database level.
* If you ever need to build similar pipelines for other forms of database, there are libraries such PyODBC which operate very similarly.
* SQL connection databases utilizes cursors for data traversal and retrieval.  This is kind of like an iterator in Python.
* Cursor operations typically goes like the following:
    - execute a query
    - fetch rows from query result if it is a SELECT query
    - because it is iterative, previously fetched rows can only be fetched again by rerunning the query
    - close cursor through .close()
* Cursors and Connections must be closed using .close() or else Postgres will lock certain operation on the database/tables to connection is severed. 

#### From Assessment 2
Given a table named universities (this is the same data as used in the Pandas challenge above) which contains university data with these columns: name, address, Website, Type, Size. Return a SQL query that gives each type and the average size of each university of that type in ascending order (same question as Pandas as well).

In [1]:
SELECT Type AS university_type, AVG(Size) AS average_size FROM universities
GROUP BY Type
ORDER BY average_size;

SyntaxError: invalid syntax (<ipython-input-1-0fffe4694e9b>, line 1)

# Warmup - Wed, 4/1
Prompt: 
A psychology researcher who does work in creativity wants to determine whether her sample of 50-year-old adults 
(N = 150) differs statistically from the population of 50-year-olds in intelligence. 
For her sample, she calculates the following: Mean = 109, SD = 10. Hint: 
By definition, the mean IQ of this population is 100 and the population standard deviation is 15.

Info:
    N = 150: 50-year-old adults
    sample Mean IQ = 109
    sample SD IQ = 10

    Pop Mean IQ = 100
    sample SD IQ = 15

Plan: 
    Do hypothesis testing and try to get a p-value?

Work:
    H0: sample mean = pop mean
    Ha: sample mean > pop mean
    alpha = 0.05

Plan B - FINAL:
CI
using CLT since N > 30:
Find 95% CI
upper CI = samp_mean + 1.96 * pop SD / n 
lower CI = samp_mean - 1.96 * pop SD / n 

if pop mean within 95% CI, sample does NOT differ statistically

# Mongo

In [ ]:
from pymongo import MongoClient
import pprint
# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)
db = client['unicorns']
unicorns = db['unicorns']

## AWS MongoDB Installation
You should already have a local Mongo docker container.  Let's practice our AWS skills by spinning up a micro instance and practice installing on a remote machine.
# NOTE!!!
`ssh my_first_ec2`
*after updating ~/.ssh/config with new DNS*
# !!!
1. To install MongoDB, use your operating system's package manager:
   - Ubuntu Linux: `sudo apt-get install mongodb`

2. Much like Postgres, you will need to launch the server before using Mongo for the first time. 

   - Ubuntu Linux: `sudo /etc/init.d/mongodb start` 

3. Check your installation by opening the MongoDB Client:

   - Open a new terminal and type `mongo` to open up a Mongo shell
   - Type `show dbs;` to show the databases you have
   - You can exit by typing `exit`

4. Resources and quick references to Mongo commands:

   - [MongoDB Cheatsheet][mongo-cheatsheet]
   - [Mongo Docs][mongo-docs]
   - [MongoDB Reference Cards][mongo-ref-cards]

# Web Scraping
## Check [website]/robots.txt to see if you "can" do web scraping/crawling

In [3]:
import warnings
warnings.filterwarnings('ignore')

from pymongo import MongoClient
import pprint

import copy
import pandas as pd

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

#### Step 1: Check out the website in a browser.
#### Step 2: Send a GET request for the data.

In [5]:
deer_tier_url = 'http://deertier.com/Leaderboard/AnyPercentRealTime'
r = requests.get(deer_tier_url)
# A status code of `200` means that everything went well.
r.status_code

200

In [6]:
r.content

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head runat="server">\r\n    \r\n    <link rel="apple-touch-icon" sizes="180x180" href="/apple-touch-icon.png" />\r\n    <link rel="icon" type="image/png" href="/favicon-32x32.png" sizes="32x32" />\r\n    <link rel="icon" type="image/png" href="/favicon-16x16.png" sizes="16x16" />\r\n    <link rel="manifest" href="/manifest.json" />\r\n    <link rel="mask-icon" href="/safari-pinned-tab.svg" color="#fec200" />\r\n\r\n    <meta name="apple-mobile-web-app-title" content="Deer Tier" />\r\n    <meta name="application-name" content="Deer Tier" />\r\n    <meta name="theme-color" content="#331f52" />\r\n    <meta name="viewport" content="width=device-width" />\r\n\r\n    <link rel="Stylesheet" type="text/css" href="/CSS/Reset.css" />\r\n    <link rel="Stylesheet" type="text/css" href="/CSS/font-awesome.min.css" />\r\n 

#### Step 3: Save all the hypertext into mongo for later use.

In [7]:
client = MongoClient('localhost', 27017)
db = client.metroid
pages = db.pages

pages.insert_one({'html': r.content})

In [11]:
pages.distinct('html')

[b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head runat="server">\r\n    \r\n    <link rel="apple-touch-icon" sizes="180x180" href="/apple-touch-icon.png" />\r\n    <link rel="icon" type="image/png" href="/favicon-32x32.png" sizes="32x32" />\r\n    <link rel="icon" type="image/png" href="/favicon-16x16.png" sizes="16x16" />\r\n    <link rel="manifest" href="/manifest.json" />\r\n    <link rel="mask-icon" href="/safari-pinned-tab.svg" color="#fec200" />\r\n\r\n    <meta name="apple-mobile-web-app-title" content="Deer Tier" />\r\n    <meta name="application-name" content="Deer Tier" />\r\n    <meta name="theme-color" content="#331f52" />\r\n    <meta name="viewport" content="width=device-width" />\r\n\r\n    <link rel="Stylesheet" type="text/css" href="/CSS/Reset.css" />\r\n    <link rel="Stylesheet" type="text/css" href="/CSS/font-awesome.min.css" />\r\n

#### Step 4: Parse the hypertext with BeautifulSoup.

In [12]:
soup = BeautifulSoup(r.content, "html")
print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head runat="server">
<link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/manifest.json" rel="manifest"/>
<link color="#fec200" href="/safari-pinned-tab.svg" rel="mask-icon"/>
<meta content="Deer Tier" name="apple-mobile-web-app-title"/>
<meta content="Deer Tier" name="application-name"/>
<meta content="#331f52" name="theme-color"/>
<meta content="width=device-width" name="viewport"/>
<link href="/CSS/Reset.css" rel="Stylesheet" type="text/css"/>
<link href="/CSS/font-awesome.min.css" rel="Stylesheet" type="text/css"/>
<link href="/CSS/Styles.css?v=30" rel="Stylesheet" type="text/css"/>
<script src="/Scripts/jquery-3.3.1.min.js" type="te

In [13]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head runat="server">
  <link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/manifest.json" rel="manifest"/>
  <link color="#fec200" href="/safari-pinned-tab.svg" rel="mask-icon"/>
  <meta content="Deer Tier" name="apple-mobile-web-app-title"/>
  <meta content="Deer Tier" name="application-name"/>
  <meta content="#331f52" name="theme-color"/>
  <meta content="width=device-width" name="viewport"/>
  <link href="/CSS/Reset.css" rel="Stylesheet" type="text/css"/>
  <link href="/CSS/font-awesome.min.css" rel="Stylesheet" type="text/css"/>
  <link href="/CSS/Styles.css?v=30" rel="Stylesheet" type="text/css"/>
  <script src="/Scripts/jq

In [14]:
print(soup.title)

<title>Any% Real Time - Deer Tier</title>


## Example: Use a web API to scrape Wikipedia

In [16]:
# Step 1: Get the Data
import json
import re
headers = {'User-Agent': 'GalvanizeDataWrangling/1.1 matthew.drury@galvanize.com'}
api_url = 'https://en.wikipedia.org/w/api.php'

# Parameters for the API request: We want the Unicorn page encoded as json.
payload = {'action': 'parse', 'format': 'json', 'page': "Unicorn"}

r = requests.post(api_url, data=payload, headers=headers)

In [17]:
print(r.json().keys())

dict_keys(['parse'])


In [18]:
print(r.json()['parse'])

{'title': 'Unicorn', 'pageid': 43236, 'revid': 947503891, 'text': {'*': '<div class="mw-parser-output"><div role="note" class="hatnote navigation-not-searchable">For other uses, see <a href="/wiki/Unicorn_(disambiguation)" class="mw-disambig" title="Unicorn (disambiguation)">Unicorn (disambiguation)</a>.</div>\n<div role="note" class="hatnote navigation-not-searchable">Not to be confused with <a href="/wiki/Unicron" title="Unicron">Unicron</a>.</div>\n<p class="mw-empty-elt">\n</p>\n<div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">Legendary creature, like a horse with a large horn projecting from forehead</div>\n<table class="infobox" style="width:22em"><caption>Unicorn</caption><tbody><tr><td colspan="2" style="text-align:center"><div class="center"><div class="floatnone"><a href="/wiki/File:DomenichinounicornPalFarnese.jpg" class="image"><img alt="DomenichinounicornPalFarnese.jpg" src="//upload.wikimedia.org/wikipedia/commons/thumb/8/82/Domenich

## Spark
 * Need parallel computing on Big Data (billions of lines of data)
 # Using Spark & PySpark with Docker
```bash
$ docker run -d --name sparkbook -p 8881:8888 -v "$PWD":/home/jovyan/work jupyter/pyspark-notebook start.sh jupyter lab --LabApp.token=''
```
* RDD: Resilient Distributed Datasets (RDD)
* DAG: Directed Acyclic Graph

In [1]:
import pyspark as ps    # for the pyspark suite
spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('[appname  of choosing]')
         .getOrCreate()
        )
sc = spark.sparkContext

ModuleNotFoundError: No module named 'pyspark'

# Spark Sql

#### `.withColumn("label", func)` : user defined function
`udf()` turns a normal python function into something Spark can parallelized across its distributed data. `udf()` requires two arguments: a function, and the data type the function will return

In [ ]:
import math
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType, FloatType

def my_specialfunc(h,l,o,c):
    return ((h-l)*(math.exp(o-c)))

my_specialfunc_udf = udf(my_specialfunc, FloatType())

df_out = df_aapl.withColumn("special", my_specialfunc_udf(df_aapl['High'], 
                                                          df_aapl['Low'], 
                                                          df_aapl['Open'], 
                                                          df_aapl['Close']))

# SQL Commands
* LIMIT:
* FETCH:


# Assessment 3

In [31]:
#1
import numpy as np
import pandas as pd
def add_column(arr, col):
    '''Return a numpy array containing arr with col added as a final column.
    You can assume that the number of rows in arr is the same as the length
    of col.

    Parameters
    ----------
    arr: NumPy Array (2-dimensional)
    col: NumPy array

    Returns
    -------
    NumPy Array (2-dimensional)

    >>>add_column(np.array([[1, 2], [3, 4]]), np.array([5, 6)))
    np.array([[1, 2, 5], [3, 4, 6]])
    '''
    return np.array(pd.concat([pd.DataFrame(arr),pd.Series(col)],axis=1))

In [32]:
add_column(np.array([[1, 2], [3, 4]]), np.array([5, 6]))

array([[1, 2, 5],
       [3, 4, 6]])

In [35]:
#2
import numpy as np
def only_positive(arr):
    '''Return a numpy array containing only the rows from arr where all
    the values in that row are positive.
    USE NUMPY METHODS, AVOID USING A FOR-LOOP.

    Parameters
    ----------
    arr: NumPy Array (2-dimensional)

    Returns
    -------
    NumPy Array (2-dimensional)

    >>>np.array([[1, -1, 2],
                    [3, 4, 2],
                    [-8, 4, -4]])
    np.array([[3, 4, 2]])
    '''
    return arr[(arr > 0).all(axis=1)]

In [36]:
print(only_positive(np.array([[1, -1, 2],
                    [3, 4, 2],
                    [-8, 4, -4]])))

[[3 4 2]]


In [5]:
#3
import pandas as pd
import numpy as np
def make_series(start, length, index):
    '''Create a pandas Series of length "length" with index "index"
    and with elements that are sequential integers starting from "start".
    You may assume the length of index will be "length".

    Parameters
    ----------
    start, length: int, int
    index: list
        Index of length "length"

    Returns
    -------
    Pandas Series

    >>>print(make_series(5, 3, ['a', 'b', 'c']))
    a    5
    b    6
    c    7
    dtype: int64
    '''
    arr = np.arange(start,start+length)
    return pd.Series(arr, index)
   

In [6]:
print(make_series(5, 3, ['a', 'b', 'c']))


a    5
b    6
c    7
dtype: int64


SQL Challenges
For the following SQL challenges you will use two tables in the database with these columns:

rent: Neighborhood, City, State, med_2011, med_2014
buy: Neighborhood, City, State, med_2011, med_2014
The values in the med_2011/med_2014 columns are integers corresponding to the price on that date.

In [ ]:
WITH join1 AS (
    SELECT rent.med_2014, buy.med_2014
    FROM rent
    FULL JOIN buy
    ON rent.Neighborhood=buy.Neighborhood
    WHERE rent.City='San Francisco') 


SELECT buy.neighborhood, rent.med_2014, buy.med_2014
FROM buy
FULL JOIN join1
ON buy.Neighborhood=join1.Neighborhood
WHERE buy.City='San Francisco'